# Asistente Legal (LangChain + LLM)

Este notebook implementa el proyecto: "Asistente legal / normativo" para consultar leyes colombianas.

Objetivo de esta sección: preparar el entorno e importar las librerías necesarias. Sigue las celdas en orden.


In [11]:
!uv add "langchain[openai]" dotenv ipywidgets chromadb faiss-cpu PyPDF2

## installar dependencias con pip
## %pip install -q "langchain[openai]" python-dotenv ipywidgets chromadb faiss-cpu PyPDF2

Resolved 138 packages in 14ms
Audited 121 packages in 27ms


In [3]:
# Imports y configuración inicial
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from os import getenv
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Prompt template de ejemplo
template = """Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

# Inicialización del LLM (no ejecutar si no tiene las variables de entorno configuradas)
llm = ChatOpenAI(
    api_key=getenv("OPENAPI_API_KEY"),
    base_url=getenv("OPENAPI_BASE_URL"),
    model="google/gemini-2.5-flash-lite-preview-09-2025",
)

# Ejemplo de uso (descomente si tiene las claves configuradas)
# question = "¿Qué equipo de la NFL ganó el Super Bowl en el año en que nació Justin Bieber?"
# print(llm.invoke(input=question, config={"prompt": prompt}))


In [4]:
# Loaders: widget FileUpload y helpers para parsear archivos subidos y leer `data/`
import ipywidgets as widgets
from IPython.display import display
import os
from io import BytesIO

# Widget para subir archivos
uploader = widgets.FileUpload(accept='.txt,.md,.pdf', multiple=True)
display(widgets.VBox([widgets.Label("Sube archivos legales (.pdf, .txt, .md)"), uploader]))

# Output para previsualización
output = widgets.Output()
display(output)


def parse_uploaded_files(uploader_widget):
    """Devuelve una lista de dicts {'filename', 'text'} extraídos de los archivos subidos."""
    docs = []
    # La estructura de uploader.value depende del frontend; manejar ambos casos
    items = getattr(uploader_widget, 'value', {}) or {}
    # En algunos entornos items es lista, en otros dict
    try:
        iterator = items.items()
    except Exception:
        # intentamos tratar como lista
        iterator = [(f.get('name', f"file_{i}"), f) for i, f in enumerate(items)]

    for name, fileinfo in iterator:
        # fileinfo puede ser dict con 'content' o directamente bytes
        content = fileinfo.get('content') if isinstance(fileinfo, dict) else fileinfo
        text = ''
        if name.lower().endswith('.pdf'):
            try:
                from PyPDF2 import PdfReader
                reader = PdfReader(BytesIO(content))
                pages = [p.extract_text() or '' for p in reader.pages]
                text = '\n'.join(pages)
            except Exception as e:
                text = f"[ERROR] No se pudo leer PDF ({name}): {e}"
        else:
            try:
                if isinstance(content, bytes):
                    text = content.decode('utf-8')
                else:
                    text = str(content)
            except Exception as e:
                text = f"[ERROR] decodificando {name}: {e}"
        docs.append({'filename': name, 'text': text})
    return docs


def on_preview_clicked(b):
    with output:
        output.clear_output()
        docs = parse_uploaded_files(uploader)
        if not docs:
            print('No se han encontrado archivos subidos.')
            return
        for d in docs:
            print(f"--- {d['filename']} ---")
            print(d['text'][:1500])
            print('\n')

preview_btn = widgets.Button(description='Previsualizar archivos')
preview_btn.on_click(on_preview_clicked)
display(preview_btn)


def load_local_docs(data_dir='data'):
    """Lee archivos de `data/` y devuelve lista de dicts {'filename','text'}"""
    docs = []
    if not os.path.exists(data_dir):
        print(f"No existe la carpeta '{data_dir}'. Crea la carpeta y añade documentos de ejemplo.")
        return docs
    for fn in sorted(os.listdir(data_dir)):
        path = os.path.join(data_dir, fn)
        if not os.path.isfile(path):
            continue
        text = ''
        if fn.lower().endswith('.pdf'):
            try:
                from PyPDF2 import PdfReader
                reader = PdfReader(path)
                pages = [p.extract_text() or '' for p in reader.pages]
                text = '\n'.join(pages)
            except Exception as e:
                text = f"[ERROR] leyendo PDF {fn}: {e}"
        else:
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    text = f.read()
            except Exception as e:
                text = f"[ERROR] leyendo {fn}: {e}"
        docs.append({'filename': fn, 'text': text})
    return docs

# Ejemplo de uso: docs = load_local_docs('data')
# print(docs[0]['text'][:500]) if docs else print('No hay documentos en data/')


Output()

Button(description='Previsualizar archivos', style=ButtonStyle())

## Sección: Loaders (carga de documentos)

En esta sección implementaremos los loaders que permiten: (1) subir archivos locales mediante un widget, y (2) cargar archivos desde la carpeta `data/` del repositorio. Más adelante convertiremos estos textos en chunks y los indexaremos en una vector DB.

Instrucciones rápidas:
- Usa el widget para subir `.pdf`, `.txt` o `.md`.
- Pulsa "Previsualizar archivos" para ver el texto extraído (o errores si falta una dependencia de PDF).

In [5]:
# Preprocesamiento: splitters y creación de chunks
# Carga documentos desde `data/`, aplica un splitter y guarda los chunks en `DOCUMENT_CHUNKS`.
from collections import Counter

# Intentar importar el splitter de LangChain; si la API cambia, manejar el fallback
try:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
except Exception:
    try:
        # versión alternativa
        from langchain.text_splitter import CharacterTextSplitter as RecursiveCharacterTextSplitter
    except Exception:
        RecursiveCharacterTextSplitter = None


def create_chunks(docs, chunk_size=1000, chunk_overlap=200):
    """Devuelve lista de dicts {'source','chunk_index','text'}"""
    if RecursiveCharacterTextSplitter is None:
        raise ImportError("No se encontró RecursiveCharacterTextSplitter. Instala langchain actualizado.")
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    all_chunks = []
    for d in docs:
        text = d.get('text') or ''
        if not text.strip():
            # texto vacío o error al leer
            continue
        texts = splitter.split_text(text)
        for i, t in enumerate(texts):
            all_chunks.append({'source': d.get('filename', 'unknown'), 'chunk_index': i, 'text': t})
    return all_chunks

# Cargar documentos locales desde data/
DOCUMENTS = load_local_docs('data')
print(f"Documentos cargados: {len(DOCUMENTS)}")

# Crear chunks
DOCUMENT_CHUNKS = create_chunks(DOCUMENTS, chunk_size=1000, chunk_overlap=200) if DOCUMENTS else []
print(f"Total chunks generados: {len(DOCUMENT_CHUNKS)}")

# Mostrar conteo por documento y un ejemplo
if DOCUMENT_CHUNKS:
    per_doc = Counter([c['source'] for c in DOCUMENT_CHUNKS])
    for doc, cnt in per_doc.items():
        print(f" - {doc}: {cnt} chunks")
    print('\nEjemplo (primer chunk, 500 chars):')
    print(DOCUMENT_CHUNKS[0]['text'][:500])
else:
    print('No hay chunks (documentos vacíos o problema al leer los archivos).')

# Ahora `DOCUMENTS` y `DOCUMENT_CHUNKS` están listos para embeddings / indexado.


Object ID 7606,0 ref repaired
Object ID 1,0 ref repaired
Object ID 2,0 ref repaired
Object ID 7582,0 ref repaired
Object ID 7584,0 ref repaired
Object ID 7583,0 ref repaired
Object ID 7589,0 ref repaired
Object ID 7591,0 ref repaired
Object ID 7590,0 ref repaired
Object ID 7596,0 ref repaired
Object ID 7598,0 ref repaired
Object ID 7597,0 ref repaired
Object ID 4,0 ref repaired
Object ID 7604,0 ref repaired
Object ID 6,0 ref repaired
Object ID 8,0 ref repaired
Object ID 10,0 ref repaired
Object ID 12,0 ref repaired
Object ID 14,0 ref repaired
Object ID 16,0 ref repaired
Object ID 18,0 ref repaired
Object ID 20,0 ref repaired
Object ID 22,0 ref repaired
Object ID 24,0 ref repaired
Object ID 26,0 ref repaired
Object ID 28,0 ref repaired
Object ID 30,0 ref repaired
Object ID 32,0 ref repaired
Object ID 34,0 ref repaired
Object ID 36,0 ref repaired
Object ID 38,0 ref repaired
Object ID 40,0 ref repaired
Object ID 42,0 ref repaired
Object ID 44,0 ref repaired
Object ID 46,0 ref repaired
Obj

Documentos cargados: 2
Total chunks generados: 1215
 - Constitución_Política_1_de_1991_Asamblea_Nacional_Constituyente.pdf: 1050 chunks
 - reglamento_academi_pregrado.pdf: 165 chunks

Ejemplo (primer chunk, 500 chars):
Departamento Administrativo de la Función Pública
Constitución Política 1 de 1991 Asamblea
Nacional Constituyente1 EVA - Gestor Normativo
Constitución Política 1 de 1991 Asamblea Nacional
Constituyente
Los datos publicados tienen propósitos exclusivamente informativos. El Departamento Administrativo de la Función Pública no se hace
responsable de la vigencia de la presente norma. Nos encontramos en un proceso permanente de actualización de los contenidos.
CONSTITUCIÓN POLITICA DE LA REPUBLICA DE


In [18]:
!uv add langchain-openai --upgrade

Resolved 138 packages in 348ms                                       
Audited 121 packages in 25ms


In [ ]:
# Embeddings + VectorStore (Chroma preferido, FAISS fallback) y pipeline RAG
import textwrap
# Requerir DOCUMENT_CHUNKS
try:
    len(DOCUMENT_CHUNKS)
except NameError:
    raise NameError('DOCUMENT_CHUNKS no está definido. Ejecuta la celda de splitters antes de esta.')

# Import embeddings
openAIEmbeddings = None
try:
    from langchain.embeddings import OpenAIEmbeddings
    openAIEmbeddings = OpenAIEmbeddings()
except Exception:
    try:
        from langchain.embeddings.openai import OpenAIEmbeddings
        openAIEmbeddings = OpenAIEmbeddings()
    except Exception:
        openAIEmbeddings = None

# Import vectorstores
Chroma = None
FAISS = None
try:
    from langchain.vectorstores import Chroma
    Chroma = Chroma
except Exception:
    try:
        from langchain.vectorstores import FAISS
        FAISS = FAISS
    except Exception:
        Chroma = None
        FAISS = None

if openAIEmbeddings is None:
    raise ImportError('No se encontró OpenAIEmbeddings. Instala/actualiza langchain o ajusta el código según la versión.')

# Preparar textos y metadatos
texts = [c['text'] for c in DOCUMENT_CHUNKS]
metadatas = [{'source': c['source'], 'chunk_index': c['chunk_index']} for c in DOCUMENT_CHUNKS]

# Inicializar embeddings
emb = openAIEmbeddings()

# Construir o cargar vectorstore
vectordb = None
try:
    if Chroma is not None:
        # persist_directory crea/usa la carpeta chroma_db
        vectordb = Chroma.from_texts(texts, embedding=emb, metadatas=metadatas, collection_name='laws', persist_directory='chroma_db')
        try:
            vectordb.persist()
        except Exception:
            pass
    else:
        # FAISS fallback (in-memory)
        from langchain.vectorstores import FAISS
        vectordb = FAISS.from_texts(texts, embedding=emb, metadatas=metadatas)
except Exception as e:
    raise RuntimeError(f'Error creando vectorstore: {e}')

print('Vectorstore preparado.')

# Helper: recuperar documentos (devuelve lista de objetos con page_content y metadata)
def retrieve(query, k=4):
    # similarity_search devuelve objetos Document o similares
    try:
        results = vectordb.similarity_search(query, k=k)
    except Exception:
        # Algunas versiones usan similarity_search_with_relevance_scores
        try:
            results = [r[0] for r in vectordb.similarity_search_with_relevance_scores(query, k=k)]
        except Exception as e:
            raise RuntimeError(f'No se pudo ejecutar la búsqueda: {e}')
    return results

# Pipeline RAG: sintetizar respuesta con LLM usando contexto recuperado

def rag_answer(query, k=4):
    docs = retrieve(query, k=k)
    if not docs:
        return {'answer': 'No se encontraron documentos relevantes.', 'citations': []}

    # Construir contexto concatenando fragmentos recuperados con metadata
    ctx_parts = []
    citations = []
    for d in docs:
        # compatibilidad: Document tiene page_content y metadata
        text = getattr(d, 'page_content', None) or d.get('text') if isinstance(d, dict) else str(d)
        meta = getattr(d, 'metadata', None) or (d if isinstance(d, dict) else {})
        src = meta.get('source', meta.get('file_name', 'unknown'))
        idx = meta.get('chunk_index')
        citations.append({'source': src, 'chunk_index': idx})
        snippet = textwrap.shorten(text, width=1000, placeholder='...')
        ctx_parts.append(f"Source: {src} (chunk {idx})\n{snippet}")

    context = '\n\n'.join(ctx_parts)

    # Prompt final
    final_prompt = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer conciso en español. Indica al final las citas como [source:filename chunk_index]."

    # Llamada al LLM (usar llm.invoke si está disponible)
    try:
        resp = llm.invoke(input=final_prompt)
        # si resp es un objeto complejo, intentar extraer texto
        answer_text = str(resp)
    except Exception:
        try:
            # fallback: si llm tiene método __call__
            answer_text = llm(final_prompt)
        except Exception as e:
            answer_text = f"[ERROR llamando al LLM: {e}]"

    return {'answer': answer_text, 'citations': citations}

# Widget simple para consultas RAG
query_box = widgets.Text(placeholder='Escribe tu pregunta sobre la ley (ej: ¿Qué artículo protege la libertad de expresión?)', description='Pregunta:', layout=widgets.Layout(width='80%'))
ask_btn = widgets.Button(description='Consultar')
out = widgets.Output()


def on_ask(b):
    with out:
        out.clear_output()
        q = query_box.value.strip()
        if not q:
            print('Escribe una pregunta primero.')
            return
        print('Buscando documentos relevantes...')
        res = rag_answer(q, k=4)
        print('\n--- RESPUESTA ---\n')
        print(res['answer'])
        print('\n--- CITAS ---')
        for c in res['citations']:
            print(f"- {c['source']} (chunk {c['chunk_index']})")

ask_btn.on_click(on_ask)

display(widgets.VBox([query_box, ask_btn, out]))

print('Interfaz RAG lista. Escribe una pregunta y pulsa "Consultar".')


ImportError: No se encontró OpenAIEmbeddings. Instala/actualiza langchain o ajusta el código según la versión.